In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [2]:
# Start an MAPDL instance
mapdl = launch_mapdl()

In [28]:
mapdl.finish()
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2024 R2          24.2     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   01:17:48  NOV 20, 2024 CP=     67.438

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

## Triangular Lattice

In [ ]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
# density = 0.5

wall_thickness = 0.5/1000
n_cell = 5
externall_wall = False
externall_wall_thickness = 0.5/1000

In [30]:
cell_size = (design_height - 2*5/6*np.sqrt(3)*wall_thickness
              - (n_cell - 1)*np.sqrt(3)*wall_thickness)/n_cell + np.sqrt(3)*wall_thickness

In [ ]:
i = 0
design_part = mapdl.rectng(0, design_width, 0, design_height)
while True:
    if i%2 == 0:
        for j in range(n_cell):
            k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = j*cell_size + 2/3*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+1)*cell_size - 1/3*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - 3/2*wall_thickness, y = (j+1/2)*cell_size + 1/6*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
            
        for j in range(n_cell-1):
            k1 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+1/2)*cell_size + 2/3*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+3/2)*cell_size - 1/3*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 + 1/2*wall_thickness, y = (j+1)*cell_size + 1/6*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
        k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 0)
        k2 = mapdl.k(x = (i+2)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 0)
        k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = 1/2*cell_size)
        hole = mapdl.a(k1, k2, k3)
        try:
            design_part = mapdl.asba(na1 = design_part, na2 = hole)
        except:
            pass
        k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height)
        k2 = mapdl.k(x = (i+2)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height)
        k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness/2, y = design_height - 1/2*cell_size)
        hole = mapdl.a(k1, k2, k3)
        try:
            design_part = mapdl.asba(na1 = design_part, na2 = hole)
        except:
            pass
    else:
        for j in range(n_cell):
            k1 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = j*cell_size + 2/3*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - wall_thickness, y = (j+1)*cell_size - 1/3*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = i*cell_size*np.sqrt(3)/2 + 1/2*wall_thickness, y = (j+1/2)*cell_size + 1/6*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
        for j in range(n_cell-1):
            k1 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+1/2)*cell_size + 2/3*np.sqrt(3)*wall_thickness)
            k2 = mapdl.k(x = i*cell_size*np.sqrt(3)/2, y = (j+3/2)*cell_size - 1/3*np.sqrt(3)*wall_thickness)
            k3 = mapdl.k(x = (i+1)*cell_size*np.sqrt(3)/2 - 3/2*wall_thickness, y = (j+1)*cell_size + 1/6*np.sqrt(3)*wall_thickness)
            hole = mapdl.a(k1, k2, k3)
            try:
                design_part = mapdl.asba(na1 = design_part, na2 = hole)
            except:
                pass
    if i*cell_size*np.sqrt(3)/2 > design_width:
        break
    i+=1

In [26]:
mapdl.aplot(cpos="xy", show_lines=True)

Widget(value='<iframe src="http://localhost:3530/index.html?ui=P_0x1545a023f50_2&reconnect=auto" class="pyvist…

In [ ]:
design_part = mapdl.rectng(0, design_width, 0, design_height)
mapdl.rectng(0, design_width, 0, design_height)

145

In [ ]:
mapdl.rectng(-total_width/2+design_width/2, 0, 0, design_height)
mapdl.rectng(total_width/2+design_width/2, 0, 0, design_height)
mapdl.aadd("all")

ValueError: Use ``input_strings`` for multi-line commands

In [57]:
mapdl.aadd("all")
mapdl.aplot(cpos="xy", show_lines=True)

MapdlExitedError: The MAPDL instance has been exited before running the command: /com, Selecting kps from an iterable (i.e. set, list, tuple, or array)

In [ ]:
design_part_box = [0, design_width, 0, design_height]
mapdl.rectng(*design_part_box)
mapdl.aina()

mapdl.rectng(-total_width/2+design_width/2, 0, 0, design_height)
mapdl.rectng(total_width/2+design_width/2, 0, 0, design_height)

design_width = 60/1000
design_height = 25/1000